In [1]:
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 2 - 2.3 Segmentação de Negócio

### Geração de Massa

In [3]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.saldoMedio':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'solicitante.flagPrincipalidadeAll':[0],
                             'solicitante.idade':[18]
                            })

In [4]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_segmentacaoNegocio = sherlock.Gera_Massa(base_entrada)
massa_segmentacaoNegocio.gera_cenario('solicitante.saldoMedio',[-99999,0,99999,100000,100001,39999,40000,40001])
massa_segmentacaoNegocio.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,11124.15,11124.16,11124.17,3442.36,3442.37,3442.38])
massa_segmentacaoNegocio.gera_cenario('solicitante.flagPrincipalidadeAll',[1,0])
massa_segmentacaoNegocio.gera_cenario('solicitante.idade',[17,18,19,20])


massa_segmentacaoNegocio.get_massa_final()

massa_segmentacaoNegocio.set_bom_arqv()
massa_segmentacaoNegocio.converte_massa()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
listaResultado = rp.execucaoModular(massa_segmentacaoNegocio.get_massa_json(),rp.SegmentacaoNegocio)

##### Valida se todas condições estão sendo abordadas

In [6]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['segmentacaoNegocio'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,Varejo+,42
1,Jovem Cliente,168
2,Varejo,12
3,Alta renda,114
4,Alta renda investidor,112


In [7]:
# Renomeação das colunas para o cabeçalho do RMA
massa_segmentacaoNegocio.rename_cabecalho()

In [8]:
massa_segmentacaoNegocio.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacaoNegocio","segmentacaoNegocio")
massa_segmentacaoNegocio.get_massa_dm()

,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.saldoMedio,rendaLiquidaPicpay,payloadParam.solicitante.flagPrincipalidadeAll,idade,result:segmentacaoNegocio,expected:segmentacaoNegocio
0,1,,,-99999,-99999.00,1,17,,Varejo+
1,2,,,-99999,-99999.00,1,18,,Jovem Cliente
2,3,,,-99999,-99999.00,1,19,,Jovem Cliente
3,4,,,-99999,-99999.00,1,20,,Varejo+
4,5,,,-99999,-99999.00,0,17,,Varejo
...,...,...,...,...,...,...,...,...,...
443,444,,,40001,3442.38,1,20,,Alta renda
444,445,,,40001,3442.38,0,17,,Alta renda
445,446,,,40001,3442.38,0,18,,Jovem Cliente
446,447,,,40001,3442.38,0,19,,Jovem Cliente


### Conversão da massa para CSV

In [9]:
massa_segmentacaoNegocio.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaSegmentacaoNegocioV2.csv',index=False)